In [42]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import os, subprocess
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Location where original video is stored and segmented video location
segment_loc = "/Users/sharingan/Documents/IEMOCAP/sentences/avi/"
original_vid_loc = "/Users/sharingan/Documents/IEMOCAP/avi/DivX/"

# Where you want to store input processed frames
frames_loc = "/Users/sharingan/Documents/IEMOCAP/frames_for_CNN/"

# Location where the datafile is located
datafile_loc = "/Users/sharingan/Documents/IEMOCAP/EmoEvaluation/"
frames_no_stripes_loc = "/Users/sharingan/Documents/IEMOCAP/Frames_no_stripes/"

In [3]:
# Select videos with script (We are not using impro videos)
all_vid_names = subprocess.check_output(["ls",original_vid_loc]).split()
all_names = [x[:-4] for x in all_vid_names]
script_vid_names = [x for x in all_names if x.split('_')[1][0] == 's'] 
print script_vid_names

['Ses01F_script01_1', 'Ses01F_script01_2', 'Ses01F_script01_3', 'Ses01F_script02_1', 'Ses01F_script02_2', 'Ses01F_script03_1', 'Ses01F_script03_2', 'Ses01M_script01_1', 'Ses01M_script01_2', 'Ses01M_script01_3', 'Ses01M_script02_1', 'Ses01M_script02_2', 'Ses01M_script03_1', 'Ses01M_script03_2']


# EXTRACTING FRAMES AND SAVING THEM IN SCRIPT-WISE FASHION

In [56]:
# creating directories for frames on the basis of scpritps
# creating subdirectories with 4 emotions

def create_directory(directory_name):
    os.system('mkdir ' + frames_loc + directory_name)

selected_scripts = ['script01', 'script02', 'script03']
# selected_scripts = ['test', 'test2']
selected_emotions = ['ang', 'hap', 'sad', 'neu']
# selected_emotions = ['ang', 'neu']
for script in selected_scripts:
    create_directory(script)
    for emo in selected_emotions:
        create_directory(script + "/" + emo)

Only needed to execute once

In [57]:
# def extract_videos_for_CNN(video_input_file_path, script, emo, segment_name):
#     print '\n Extracting frames from video: ', video_input_file_path
#     vidcap = cv2.VideoCapture(video_input_file_path)
#     success, image = vidcap.read()
#     features = []
#     success = True
#     count = 0
#     while success:
# #         plt.imshow(image, cmap='gray')
# #         plt.xticks([]),plt.yticks([])  # to hide tick values on X and Y axis
# #         plt.show()
#         success, image = vidcap.read()
# #         print('Read a new frame: ', success)
#         if success:
#             image = image[120:360, 0:720]
#             image = cv2.resize(image, (480, 480), interpolation=cv2.INTER_AREA)
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# #             channels = image.shape[2]
# #             print image.shape
# #             for channel in range(channels):
# #                 features.append(image[:, :, channel])
#             count = count + 1
#             loc = frames_loc + script + '/' + emo + '/' + segment_name + '_' + str(count) + '.jpg'
#             print cv2.imwrite(loc, image),
        

# for name in script_vid_names:
# #     print "\n Extracting frames from ", name
#     txt_file_name = name + '.txt'
#     with open(datafile_loc + txt_file_name) as f:
#         data = f.readlines()
#     data = iter(data)
#     try:
#         for line in data:
#             if line != '\n':
#                 continue;
#             line = next(data)
#             segment_name = line.split()[3]
#             emotion = line.split()[4]
#             script = segment_name.split('_')[1]
#             this_segment_loc = segment_loc + name + '/' + segment_name + '.avi'
#             if emotion in selected_emotions:
#                 extract_videos_for_CNN(this_segment_loc, script, emotion, segment_name)
            
#     except(StopIteration):
#         pass

# CNN

In [38]:
#batch size
bs = 32

In [44]:
def CNN_model():
    model = Sequential()
    # define CNN model
    model.add(Conv2D(bs, (3, 3), activation = 'relu' ,input_shape = (480,480,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(len(selected_emotions), activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model
# print model.summary()

In [45]:
def generate_train(directory):
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)

    train_generator = train_datagen.flow_from_directory(directory,
                                                        target_size=(480, 480),
                                                        batch_size=bs,
                                                        class_mode='categorical',
                                                        color_mode = 'grayscale')
    return train_generator

def generate_test(directory):
    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(directory,
                                                            target_size=(480, 480),
                                                            batch_size=bs,
                                                            class_mode='categorical',
                                                            color_mode='grayscale')
    return test_generator

In [55]:
# create training and testing set

kf = KFold(n_splits=len(selected_scripts))
kf = kf.split(selected_scripts)

# Each iteration represents one fold
for train, test in kf:
    print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    # New model for every fold
    model = CNN_model()
    # Fit training data
    for t in train:
        train_generator = generate_train(frames_loc + selected_scripts[t])
        
        model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.samples / train_generator.batch_size,
            epochs=5)
    
    # Test data
    for t in test:
        test_generator = generate_test(frames_loc + selected_scripts[t])
    
    
    class_labels = list(test_generator.class_indices.keys())
    predictions = model.predict_generator(test_generator, np.math.ceil(float(test_generator.samples)/float(test_generator.batch_size)))
    predicted_classes = np.argmax(predictions, axis=1)
    
    # results
    report = metrics.classification_report(test_generator.classes, predicted_classes, target_names=class_labels)
    
    # confusion matrix
    confusion_mat = metrics.confusion_matrix(test_generator.classes, predicted_classes)
    
    print report  
    print class_labels
    print confusion_mat

#     # making directories and storing training and testing samples
#     os.system("rm -rf train")
#     os.system("rm -rf test")
#     os.system("mkdir train")
#     os.system("mkdir test")
    
#     print "Creating TRAINING SET"
#     dest = " ./train/"
#     for dr in train:
#         src = frames_loc + selected_scripts[dr] + '/'
#         print os.system("rsync -a " + src + dest)
    
#     print "Creating TESTING SET"
#     dest = " ./test/"
#     for dr in test:
#         src = frames_loc + selected_scripts[dr] + '/'
#         print os.system("rsync -a " + src + dest)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
test2
test
Found 14 images belonging to 2 classes.
Epoch 1/5
14/14 [==============================] - 2s 151ms/step - loss: 8.0923 - acc: 0.5000
Epoch 2/5
14/14 [==============================] - 1s 69ms/step - loss: 8.0590 - acc: 0.5000
Epoch 3/5
14/14 [==============================] - 1s 69ms/step - loss: 8.0590 - acc: 0.5000
Epoch 4/5
14/14 [==============================] - 1s 70ms/step - loss: 8.0590 - acc: 0.5000
Epoch 5/5
14/14 [==============================] - 1s 69ms/step - loss: 8.0590 - acc: 0.5000
Found 14 images belonging to 2 classes.


/anaconda2/envs/ds27/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         ang       0.50      1.00      0.67         7
         neu       0.00      0.00      0.00         7

   micro avg       0.50      0.50      0.50        14
   macro avg       0.25      0.50      0.33        14
weighted avg       0.25      0.50      0.33        14

['ang', 'neu']
[[7 0]
 [7 0]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
test
test2
Found 14 images belonging to 2 classes.
Epoch 1/5
14/14 [==============================] - 2s 158ms/step - loss: 0.5535 - acc: 0.5714
Epoch 2/5
14/14 [==============================] - 1s 73ms/step - loss: 1.1167 - acc: 0.4286
Epoch 3/5
14/14 [==============================] - 1s 73ms/step - loss: 0.6752 - acc: 0.5714
Epoch 4/5
14/14 [==============================] - 1s 73ms/step - loss: 0.8017 - acc: 0.5714
Epoch 5/5
14/14 [==============================] - 1s 73ms/step - loss: 0.8375 - acc: 0.5000
